In [1]:
import os
import sys
from functools import lru_cache

import pandas as pd
from loguru import logger

sys.path.insert(0, r'C:\Users\user\Desktop\github\sad_dev')
from source.core.de.connectors import PGConnector

In [2]:
logger.remove()
logger.add(
    sink=sys.stderr,
    level='TRACE',
)

1

In [3]:
ROOT_FOLDER: str = r'C:\Users\user\YandexDisk\batch_movement\reference'
FILENAME: str = 'nomenclature.xlsx'

In [4]:
ENGINE = PGConnector().engine

2025-05-15 16:54:15.898 | INFO     | source.core.de.connectors.postgresql:__create_engine:92 - Начало создания подключения к базе данных.
2025-05-15 16:54:15.899 | INFO     | source.core.de.connectors.postgresql:__create_connection_string:59 - Начало формирования строки подключения к базе данных.
2025-05-15 16:54:15.901 | ERROR    | source.core.de.connectors.postgresql:__create_connection_string:76 - Непредвиденная ошибка при создании строки подключения к базе данных: 'postgresql'
2025-05-15 16:54:15.901 | ERROR    | source.core.de.connectors.postgresql:__create_engine:100 - Непредвиденная ошибка при создании подключения к базе данных: Непредвиденная ошибка при создании строки подключения к базе данных


ValueError: Непредвиденная ошибка при создании подключения к базе данных

In [ ]:
@lru_cache(maxsize=32)
def create_dataframe() -> pd.DataFrame:
    dataframe: pd.DataFrame = pd.read_excel(
        io=os.path.join(ROOT_FOLDER, FILENAME),
        engine='openpyxl',
        dtype='string'
    )
    return dataframe

In [ ]:
dataframe: pd.DataFrame = create_dataframe()

In [ ]:
dataframe.info(memory_usage='deep')

In [ ]:
# Замена пропусков на строки '_нет данных';
dataframe.fillna('_нет данных', inplace=True)

In [ ]:
dataframe.info(memory_usage='deep')

In [ ]:
for column in dataframe.columns:
    try:
        logger.info(f'Преобразование строк столбца "{column}" к нижнему регистру;')
        dataframe[column] = dataframe[column].str.lower()
        logger.success(f'Преобразование строк столбца "{column}" к нижнему регистру выполнен;')
    
    except Exception as e:
        logger.error(f'Непредвиденная ошибка: {e};')

In [ ]:
dataframe.info(memory_usage='deep')

In [ ]:
# Расчёт половины строк в DataFrame;
df_thrd_part = dataframe.shape[0] // 3
logger.trace(f'Количество строк в DataFrame: {dataframe.shape[0]}; Треть: {df_thrd_part};')

# Определение необходимости и преобразование типов данных к категориальному;
for column in dataframe.columns:
    logger.trace(f'Тип данных для столбца "{column}": {dataframe[column].dtype}.')
    
    # Определение количества уникальных значений в выбранном столбце;
    qnt_unq_val = dataframe[column].nunique()
    logger.trace(f'Количество уникальных значений: {qnt_unq_val}.')
    
    # Если количество уникальных значений менее половины строк датафрейма, то преобразовать столбец к категориальному типу данных.
    if qnt_unq_val <= df_thrd_part:
        logger.debug('Количество уникальных значений менее трети строк датафрейма;')
        logger.info(f'Преобразование столбца "{column}" к категориальному типу данных;')
        # Преобразование типа данных к категориальному;
        try:
            dataframe[column] = dataframe[column].astype(dtype='category')
            logger.success(f'Столбец "{column}" преобразован к категориальному типу данных;')
        except Exception as e:
            logger.error(f'Непредвиденная ошибка: {e};')

    # Если количество уникальных значений больше половины строк датафрейма, то преобразование типа данных не требуется.
    else:
        logger.debug('Количество уникальных значений больше трети строк датафрейма;')
        logger.info(f'Преобразование типов данных для столбца "{column}" не требуется;')

In [ ]:
dataframe.info(memory_usage='deep')

In [ ]:
# Получить текущие данные из базы данных PostgreSQL;
query: str = """
SELECT
    *
FROM
    constant.nomenclature
"""

In [ ]:
with 

In [ ]:
dataframe